In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [16]:
df = pd.read_csv('../data/total_data_set.csv')
df

,날짜,종가,오픈,거래량,변동 %,실업자,실업률(%),청년실업자,청년실업률(%),빈도(부정),빈도(긍정),코스피지수변화,순매수비율
0,2017-05-23,"3,206,000","3,104,000",21.58K,3.29%,99.5,3.6,40.8,9.2,16.90149,14.58574,0.0033,0.008202
1,2017-05-24,"4,175,000","3,206,000",34.68K,30.22%,99.5,3.6,40.8,9.2,16.92568,15.11493,0.0024,0.008202
2,2017-05-25,"4,199,000","4,175,000",35.91K,0.57%,99.5,3.6,40.8,9.2,16.57495,16.02447,0.0110,0.008202
3,2017-05-26,"3,227,000","4,199,000",36.65K,-23.15%,99.5,3.6,40.8,9.2,14.21660,12.58475,0.0053,0.008202
4,2017-05-27,"3,152,000","3,227,000",33.75K,-2.32%,99.5,3.6,40.8,9.2,10.08647,8.97965,0.0000,0.008202
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1501,2021-07-02,"39,417,000","39,335,000",0.52K,0.21%,92.0,3.2,30.8,7.2,16.68379,15.77641,-0.0001,0.002991
1502,2021-07-03,"40,335,000","39,413,000",0.19K,2.33%,92.0,3.2,30.8,7.2,13.23093,10.73259,0.0000,0.002991
1503,2021-07-04,"40,914,000","41,183,000",0.55K,1.44%,92.0,3.2,30.8,7.2,11.24750,10.78220,0.0000,0.002991
1504,2021-07-05,"39,484,000","40,914,000",0.70K,-3.50%,92.0,3.2,30.8,7.2,15.66184,17.03323,0.0035,0.002991


In [ ]:
df['거래량'] = df.거래량.str.replace('K', '').astype('float32')


In [ ]:
df['변동 %']=df.거래량.replace('%', '').astype('float32')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df=df.drop('날짜', axis=1)
df_scaled = scaler.fit_transform(df)

df= pd.DataFrame(df_scaled)




In [ ]:
df.columns = ['종가', '오픈', '거래량','변동 %', '실업자','실업률(%)' ,'청년실업자', '청년실업률(%)', '빈도(부정)','빈도(긍정)', '코스피지수변화', '순매수비율']

In [ ]:
prediction_days = 121
df_train= df[:1506-prediction_days]
df_test= df[1506-prediction_days:]


In [ ]:
ax = df_train.plot()
df_test.plot(ax=ax)
plt.legend(['train', 'test'])

In [ ]:
df

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
def make_dataset(data, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)


In [ ]:
print(type(df_train))
print(X_train.shape)
X_train=df_train.drop('종가', axis=1)




In [ ]:
y_train = df_train[['종가']]


In [ ]:
y_train

In [ ]:
print(type(X_train))
print(X_train.shape)
print(y_train.shape)
print(type(y_train))

In [ ]:
def make_dataset(data, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)


In [ ]:
train_feature, train_label = make_dataset(X_train, y_train, 5)

# train, validation set 생성
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)



In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Initialising the LSTM
model = Sequential()
model.add(LSTM(16, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )
model.add(Dense(1))

# Compiling the LSTM
model.compile(loss = 'mean_squared_error',optimizer = 'adam',)
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)


# Fitting the LSTM to the Training set
model.fit(X_train, y_train, batch_size = 16, epochs = 200)


In [ ]:
X_test=df_test.drop('종가', axis=1)

y_test = df_test[['종가']]


print(X_test.shape)
print(y_test.shape)

print(type(y_test))


In [ ]:
test_feature, test_label = make_dataset(X_test, y_test,5)
test_feature.shape, test_label.shape

In [ ]:
pred = model.predict(test_feature)

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(test_label, label='actual')
plt.plot(pred, label='prediction')
plt.legend()
plt.show()

